In [1]:
import pandas as pd
import re
# Read the CSV file into a DataFrame
df = pd.read_csv('/content/Licensing_2011_with_Labels.csv')

# Define a regular expression pattern to match dates in the text.
date_pattern = r'\d{1,2} \w+ \d{4}'



# Initialize an empty list to store extracted dates.
dates = []

# Iterate through each row in the DataFrame.
for index, row in df.iterrows():
    # Extract dates from the "text" column using regular expressions.
    text = row['Content']
    match = re.search(date_pattern, text)

    if match:
        extracted_date = match.group()
        dates.append(extracted_date)
    else:
        dates.append(None)  # If no date is found, append None to the list.

# Add the extracted dates as a new "date" column in the DataFrame.
df['date'] = dates



In [2]:
df.head()

,Unnamed: 0,Document_id,Content,clean_text,Tickers,nTickers,Predictions,date
0,0.0,Document BWR0000020110727e77r000l9,Dyax Corp. Announces Second Quarter 2011 Finan...,announces second quarter 2011 financial resul...,"{'WIRE', 'HAE', 'TDW', 'HDYNQ', 'PFE', 'INTT',...",9.0,0.0,27 July 2011
1,1.0,Document BWR0000020110311e73b0001g,Acacia Research To Present At 13th Annual Cred...,"-- --march 11, 2011-- acacia research corpor...","{'WIRE', 'PDT', 'ACTG'}",3.0,0.0,11 March 2011
2,2.0,Document BWR0000020110331e73v000ag,"Vitro Out-licenses Patents, Products & Technol...","vitro out-licenses patents, products & technol...","{'WIRE', '5938B', 'TM', 'EMD', 'VODG'}",5.0,0.0,31 March 2011
3,3.0,Document BWR0000020110512e75c0007r,Navy Mutual Aid Association Selects Accenture ...,accenture to license and implement its li...,"{'WIRE', 'ACN'}",2.0,0.0,12 May 2011
4,4.0,Document PRN0000020110831e78v000na,Cadence Pharmaceuticals to Present at Three In...,"31, 2011 /prnewswire/ -- cadence pharmaceutic...","{'PRN', 'UBS', 'JMP', 'SAN', 'CADX'}",5.0,0.0,31 August 2011


In [3]:
pip install fuzzywuzzy


In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from fuzzywuzzy import fuzz
from sklearn.cluster import KMeans

# Extract the 'content' column
contents = df['Content']

# Preprocess the text data if necessary (e.g., remove punctuation and lowercase)
# For simplicity, we'll assume the text is already preprocessed

# Initialize a DataFrame to store similarity scores
similarity_df = pd.DataFrame(columns=['Content', 'Similarity'])

# Iterate through pairs of text and calculate fuzzy similarity
for i in range(len(contents)):
    row_scores = []
    for j in range(len(contents)):
        if i == j:
            similarity = 100  # 100 for identical text (maximum similarity)
        else:
            similarity = fuzz.ratio(contents[i], contents[j])
        row_scores.append(similarity)

    # Add the similarity scores to the DataFrame
    similarity_df = similarity_df.append({'Content': contents[i], 'Similarity': row_scores}, ignore_index=True)

# Merge the similarity DataFrame with the original DataFrame using an index
df = df.merge(similarity_df, left_index=True, right_index=True)

# Print the DataFrame with the similarity scores
print(df)


/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
<ipython-input-4-2c9b51b97f7b>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similarity_df = similarity_df.append({'Content': contents[i], 'Similarity': row_scores}, ignore_index=True)
<ipython-input-4-2c9b51b97f7b>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similarity_df = similarity_df.append({'Content': contents[i], 'Similarity': row_scores}, ignore_index=True)
<ipython-input-4-2c9b51b97f7b>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

     Unnamed: 0                         Document_id  \
0           0.0  Document BWR0000020110727e77r000l9   
1           1.0  Document BWR0000020110311e73b0001g   
2           2.0  Document BWR0000020110331e73v000ag   
3           3.0  Document BWR0000020110512e75c0007r   
4           4.0  Document PRN0000020110831e78v000na   
..          ...                                 ...   
343       342.0  Document PRN0000020110128e71s003sh   
344       343.0  Document PRN0000020110629e76t003uy   
345       344.0  Document PRN0000020111025e7ap0002z   
346       345.0  Document BWR0000020110228e72s0000g   
347       346.0  Document PRN0000020110629e76t004q5   

                                             Content_x  \
0    Dyax Corp. Announces Second Quarter 2011 Finan...   
1    Acacia Research To Present At 13th Annual Cred...   
2    Vitro Out-licenses Patents, Products & Technol...   
3    Navy Mutual Aid Association Selects Accenture ...   
4    Cadence Pharmaceuticals to Present at Three 

<ipython-input-4-2c9b51b97f7b>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similarity_df = similarity_df.append({'Content': contents[i], 'Similarity': row_scores}, ignore_index=True)


In [11]:
import pandas as pd
from fuzzywuzzy import fuzz

# Assuming you have already generated the 'similarity_df' and merged it with 'df' as shown in your previous code

# Iterate through pairs of text and identify rows with 100% similarity
rows_to_remove = set()
for i in range(len(contents)):
    for j in range(len(contents)):
        if i != j and df['Similarity'][i][j] == 100:
            rows_to_remove.add(i)

# Remove rows with 100% similarity from the DataFrame
df_no_100_similarity = df.drop(index=list(rows_to_remove))

# Print the DataFrame without rows with 100% similarity
print("DataFrame without 100% similarity:")
print(df_no_100_similarity)


DataFrame without 100% similarity:
     Unnamed: 0                         Document_id  \
0           0.0  Document BWR0000020110727e77r000l9   
1           1.0  Document BWR0000020110311e73b0001g   
3           3.0  Document BWR0000020110512e75c0007r   
4           4.0  Document PRN0000020110831e78v000na   
5           5.0  Document BWR0000020110125e71p00048   
..          ...                                 ...   
343       342.0  Document PRN0000020110128e71s003sh   
344       343.0  Document PRN0000020110629e76t003uy   
345       344.0  Document PRN0000020111025e7ap0002z   
346       345.0  Document BWR0000020110228e72s0000g   
347       346.0  Document PRN0000020110629e76t004q5   

                                             Content_x  \
0    Dyax Corp. Announces Second Quarter 2011 Finan...   
1    Acacia Research To Present At 13th Annual Cred...   
3    Navy Mutual Aid Association Selects Accenture ...   
4    Cadence Pharmaceuticals to Present at Three In...   
5    Acacia Re

In [13]:
df.head(1)

,Unnamed: 0,Document_id,Content_x,clean_text,Tickers,nTickers,Predictions,date,Content_y,Similarity
0,0.0,Document BWR0000020110727e77r000l9,Dyax Corp. Announces Second Quarter 2011 Finan...,announces second quarter 2011 financial resul...,"{'WIRE', 'HAE', 'TDW', 'HDYNQ', 'PFE', 'INTT',...",9.0,0.0,27 July 2011,Dyax Corp. Announces Second Quarter 2011 Finan...,"[100, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2..."


In [15]:
import pandas as pd
from fuzzywuzzy import fuzz

# Assuming you have already generated the 'similarity_df' and merged it with 'df' as shown in your previous code

# Define a function to find pairs with similarity scores between 80% and 99%
def find_similar_pairs(similarity_df, threshold_low=80, threshold_high=99):
    similar_pairs = []

    for i in range(len(similarity_df)):
        for j in range(len(similarity_df)):
            if i != j and threshold_low <= similarity_df['Similarity'][i][j] <= threshold_high:
                similar_pairs.append((i, j, similarity_df['Similarity'][i][j]))

    return similar_pairs

# Find pairs with similarity scores between 80% and 99%
pairs_between_80_and_99 = find_similar_pairs(df, threshold_low=80, threshold_high=99)

# Create a DataFrame to store the results
result_df = pd.DataFrame(columns=['Content_1', 'Content_2', 'Row_1_Date', 'Row_2_Date', 'Similarity'])

# Populate the result DataFrame
for pair in pairs_between_80_and_99:
    index_1, index_2, similarity_score = pair
    row_1_content = df['Content_x'][index_1]
    row_2_content = df['Content_x'][index_2]
    row_1_date = df['date'][index_1]
    row_2_date = df['date'][index_2]

    result_df = result_df.append({
        'Content_1': row_1_content,
        'Content_2': row_2_content,
        'Row_1_Date': row_1_date,
        'Row_2_Date': row_2_date,
        'Similarity': similarity_score
    }, ignore_index=True)

# Print the DataFrame with pairs having similarity scores between 80% and 99%
print(result_df)


                                            Content_1  \
0   Acacia Research To Present At 13th Annual Cred...   
1   Acacia Research To Present At 13th Annual Cred...   
2   Acacia Research To Present At 13th Annual Cred...   
3   Acacia Research Corporation Moves Patent  Lice...   
4   Acacia Research Corporation Moves Patent  Lice...   
5   Acacia Research Corporation Moves Patent  Lice...   
6   Acacia Research Corporation Moves Patent  Lice...   
7   Acacia Research Corporation Moves Patent  Lice...   
8   Acacia Subsidiary Enters into  License  and Se...   
9   Acacia Subsidiary Enters into  License  and Se...   
10  Acacia Subsidiary Enters into  License  and Se...   
11  Acacia Subsidiary Enters into  License  and Se...   
12  Acacia Subsidiary Enters into  License  and Se...   
13  Acacia Subsidiary Enters into  License  and Se...   
14  Acacia Subsidiary Enters into Settlement and P...   
15  Acacia Subsidiary Enters into Settlement and P...   
16  Acacia Subsidiary Enters in

<ipython-input-15-2c689536c5fc>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
<ipython-input-15-2c689536c5fc>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
<ipython-input-15-2c689536c5fc>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
<ipython-input-15-2c689536c5fc>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
<ipython-input-15-2c689536c5fc>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df

In [17]:
result_df.head(5)

,Content_1,Content_2,Row_1_Date,Row_2_Date,Similarity
0,Acacia Research To Present At 13th Annual Cred...,Acacia Subsidiary Enters into Settlement and ...,11 March 2011,13 September 2011,80
1,Acacia Research To Present At 13th Annual Cred...,Acacia Subsidiary Enters into Settlement and ...,11 March 2011,19 October 2011,80
2,Acacia Research To Present At 13th Annual Cred...,Acacia Subsidiary Enters Into Settlement and ...,11 March 2011,15 December 2011,80
3,Acacia Research Corporation Moves Patent Lice...,Acacia Subsidiary Enters into License and Se...,25 January 2011,8 March 2011,81
4,Acacia Research Corporation Moves Patent Lice...,Acacia Subsidiary Enters into Settlement and P...,25 January 2011,9 February 2011,85


In [18]:
import pandas as pd
from fuzzywuzzy import fuzz
from datetime import datetime

# Assuming you have already generated the 'similarity_df' and merged it with 'df' as shown in your previous code

# Define a function to find pairs with similarity scores between 80% and 99%
def find_similar_pairs(similarity_df, threshold_low=80, threshold_high=99):
    similar_pairs = []

    for i in range(len(similarity_df)):
        for j in range(len(similarity_df)):
            if i != j and threshold_low <= similarity_df['Similarity'][i][j] <= threshold_high:
                similar_pairs.append((i, j, similarity_df['Similarity'][i][j]))

    return similar_pairs

# Find pairs with similarity scores between 80% and 99%
pairs_between_80_and_99 = find_similar_pairs(df, threshold_low=80, threshold_high=99)

# Create a DataFrame to store the results
result_df = pd.DataFrame(columns=['Row_1_Content', 'Row_2_Content', 'Row_1_Date', 'Row_2_Date', 'Similarity', 'Date_Difference'])

# Populate the result DataFrame
for pair in pairs_between_80_and_99:
    index_1, index_2, similarity_score = pair
    row_1_content = df['Content_x'][index_1]
    row_2_content = df['Content_x'][index_2]
    row_1_date = df['date'][index_1]
    row_2_date = df['date'][index_2]

    # Convert string dates to datetime objects
    date_1 = datetime.strptime(row_1_date, '%d %B %Y')
    date_2 = datetime.strptime(row_2_date, '%d %B %Y')

    # Calculate the date difference
    date_difference = abs((date_2 - date_1).days)

    result_df = result_df.append({
        'Row_1_Content': row_1_content,
        'Row_2_Content': row_2_content,
        'Row_1_Date': row_1_date,
        'Row_2_Date': row_2_date,
        'Similarity': similarity_score,
        'Date_Difference': date_difference
    }, ignore_index=True)

# Print the DataFrame with pairs having similarity scores between 80% and 99%
print(result_df)

# Write the DataFrame to a CSV file
result_df.to_csv("result_dataframe.csv", index=False)


                                        Row_1_Content  \
0   Acacia Research To Present At 13th Annual Cred...   
1   Acacia Research To Present At 13th Annual Cred...   
2   Acacia Research To Present At 13th Annual Cred...   
3   Acacia Research Corporation Moves Patent  Lice...   
4   Acacia Research Corporation Moves Patent  Lice...   
5   Acacia Research Corporation Moves Patent  Lice...   
6   Acacia Research Corporation Moves Patent  Lice...   
7   Acacia Research Corporation Moves Patent  Lice...   
8   Acacia Subsidiary Enters into  License  and Se...   
9   Acacia Subsidiary Enters into  License  and Se...   
10  Acacia Subsidiary Enters into  License  and Se...   
11  Acacia Subsidiary Enters into  License  and Se...   
12  Acacia Subsidiary Enters into  License  and Se...   
13  Acacia Subsidiary Enters into  License  and Se...   
14  Acacia Subsidiary Enters into Settlement and P...   
15  Acacia Subsidiary Enters into Settlement and P...   
16  Acacia Subsidiary Enters in

<ipython-input-18-ffee93aab7d0>:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
<ipython-input-18-ffee93aab7d0>:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
<ipython-input-18-ffee93aab7d0>:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
<ipython-input-18-ffee93aab7d0>:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
<ipython-input-18-ffee93aab7d0>:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df

In [19]:
result_df.head(5)

,Row_1_Content,Row_2_Content,Row_1_Date,Row_2_Date,Similarity,Date_Difference
0,Acacia Research To Present At 13th Annual Cred...,Acacia Subsidiary Enters into Settlement and ...,11 March 2011,13 September 2011,80,186
1,Acacia Research To Present At 13th Annual Cred...,Acacia Subsidiary Enters into Settlement and ...,11 March 2011,19 October 2011,80,222
2,Acacia Research To Present At 13th Annual Cred...,Acacia Subsidiary Enters Into Settlement and ...,11 March 2011,15 December 2011,80,279
3,Acacia Research Corporation Moves Patent Lice...,Acacia Subsidiary Enters into License and Se...,25 January 2011,8 March 2011,81,42
4,Acacia Research Corporation Moves Patent Lice...,Acacia Subsidiary Enters into Settlement and P...,25 January 2011,9 February 2011,85,15
